In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [0]:
import os
cwd = os.getcwd()
cwd

'/Users/anirudhchatty/Desktop/yelp_dataset'

In [0]:
import pandas as pd
import json

In [0]:
# with open('/content/gdrive/My Drive/242Yelp/json upload/business.json', 'r') as f:
#    #b = json.load(f)
#   pd.read_json('/content/gdrive/My Drive/242Yelp/json upload/business.json', lines=True)

Relevant features from business dataset: address, business_id, city, categories, latitude, longitude, name, review_count, postal_code, stars

In [0]:
# pd.read_json('/content/gdrive/My Drive/242Yelp/csv upload/review.json', lines = True, chunksize = 10**5)

In [0]:
b = pd.read_json(Users/anirudhchatty/Desktop/yelp_datasetbusiness.json, lines=True, chunksize = 10**5)

NameError: name 'Users' is not defined

In [0]:
for i, chunk in enumerate(b):
  print(chunk)
  chunk.to_csv('/content/gdrive/My Drive/242Yelp/csv_upload/business_'+i+'.json', rows = False)


ValueError: ignored

strictness vs lenientness of the user in percentage (Jinlin)
ave number of review of the reataurants reviewed (Jinlin)

In [0]:
b1 = next(enumerate(b))

ValueError: ignored

In [0]:
review = pd.read_csv('/content/gdrive/My Drive/242Yelp/csv_upload/review_chunk.csv')
business = pd.read_csv('/content/gdrive/My Drive/242Yelp/csv_upload/Business_data.csv')
user = pd.read_csv('/content/gdrive/My Drive/242Yelp/csv_upload/user_chunk.csv')

review2 = pd.read_csv('/content/gdrive/My Drive/242Yelp/csv_upload/review_chunk2.csv')
#business = pd.read_csv('/content/gdrive/My Drive/242Yelp/csv_upload/Business_data.csv')
user2 = pd.read_csv('/content/gdrive/My Drive/242Yelp/csv_upload/user_chunk2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (57,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
user_trim = user[['user_id', 'review_count','average_stars', 'yelping_since']]
business_trim = business[['business_id', 'city', 'review_count', 'stars']]
review_trim = review[['user_id','business_id','text', 'date', 'stars']]

user_trim2 = user2[['user_id', 'review_count','average_stars', 'yelping_since']]
#business_trim = business[['business_id', 'city', 'review_count', 'stars']]
review_trim2 = review2[['user_id','business_id','text', 'date', 'stars']]

In [0]:
user_trim = pd.concat([user_trim,user_trim2])
reivew_trim = pd.concat([review_trim, review_trim2])

In [0]:
merge1 = user_trim.set_index('user_id').join(review_trim.set_index('user_id'), how = 'left')

In [0]:
merge1_wo_na = merge1[-merge1.business_id.isnull()]

In [0]:
merge2 = merge1_wo_na.reset_index().set_index('business_id').join(business_trim.set_index('business_id'), how = 'left', rsuffix = '_business').reset_index()#.set_index('user_id')
#merge2 = merge1_wo_na.join(business_trim, how = 'left', on = 'business_id',  rsuffix='_business')

In [0]:
merge2.shape

(46125, 11)

In [0]:
f = pd.DataFrame()

In [0]:

### LATEST FUNCTIONS TO COMPUTE FEATURES: 
def compute_lenientness(merged_df, feature_df):
  merged_df['lenient'] = 1*(merged_df.stars >= merged_df.stars_business)
  feature_df['num_lenient'] = merged_df.groupby(['user_id']).lenient.sum()
  num_reviews = merged_df.groupby('user_id').count().text ## take any column such as 'text'
  feature_df['num_reviews'] = num_reviews
  feature_df['lenientness'] = feature_df.num_lenient/feature_df.num_reviews
  return feature_df.drop(['num_lenient'], axis = 1)

def compute_pop_business_review(merged_df, feature_df, popular_threshold = 100):
  merged_df['pop_business'] = 1*merged_df.review_count_business >= popular_threshold
  feature_df['num_pop_business'] = merged_df.groupby('user_id').pop_business.sum()
  feature_df['review_pop_business_ratio'] = feature_df.num_pop_business/feature_df.num_reviews
  return feature_df.drop(['num_pop_business'],axis = 1)

def compute_word_count(merged_df,feature_df):
  merged_df['word_count'] = merged_df.text.apply(lambda x: len(list(x.split(" "))))
  feature_df['total_word_count']  = merged_df.groupby(['user_id']).word_count.sum()
  feature_df['review_word_count'] = feature_df.total_word_count/feature_df.num_reviews
  feature_df['review_word_count_percentile'] = feature_df['review_word_count'].apply(lambda x: percentileofscore(feature_df.review_word_count, x)/100)
  return feature_df.drop(['total_word_count', 'review_word_count'], axis = 1)

def compute_local_ratio(merged_df, feature_df):
  merged_df['LV'] = ((merged_df.city == 'Las vegas') | (merged_df.city == 'Las Vegas'))*1
  feature_df['num_LV'] = merged_df.groupby('user_id').LV.sum()
  feature_df['LV_local_ratio'] = feature_df.num_LV/feature_df.num_reviews
  return feature_df.drop(['num_LV'], axis = 1)

import datetime
from scipy.stats import percentileofscore
def normalize_review(merged_df, feature_df):
    feature_df['ave_review'] = merged_df.average_stars/5
    merged_df['yelping_date'] = merged_df.yelping_since.apply(lambda x: datetime.datetime.strptime(x[:10], "%Y-%m-%d"))
    feature_df['yelp_history'] =  merged_df.groupby('user_id').yelping_date.apply(lambda x: datetime.date.today() - x)
    feature_df['review_count_norm'] = feature_df.num_reviews/feature_df.yelp_history
    feature_df['review_freq'] = feature_df.review_count_norm.apply(lambda x: percentileofscore(feature_df.review_count_norm, x)/100)
    #[percentileofscore(data, i) for i in data]
    return feature_df.drop(['yelp_history', 'review_count_norm'], axis = 1)

In [0]:
f = pd.DataFrame()
feature_func_ls = [compute_lenientness, compute_pop_business_review,compute_word_count,compute_local_ratio]
for func in feature_func_ls:
  f = func(merge2, f)

(29602, 3)
(29602, 5)
(29602, 7)
(29602, 9)


In [0]:
f.head()

""


In [0]:
f2 = f.drop(columns = ['num_lenient', 'num_pop_business', 'num_reviews','total_word_count', 'num_LV'])

In [0]:
with open('/content/gdrive/My Drive/242Yelp/csv_upload/features_jinlin2.csv', 'w') as f:
  f2.to_csv(f)
#!cat /content/gdrive/My\ Drive/foo.txt

In [0]:
f2.to_csv('/content/gdrive/My Drive/242Yelp/csv_upload/features_jinlin.csv')